**GET DATA**

Summary of this notebook: ...

Todo:

1. Self supervised representation learning using colorization as a pretext task
2. Taking in an image I , rotating it by an a set angle A--> rotated image (I_A), rotated angle A (actual label) (Done)
3. Dataloader that does this somehow (Done)
4. Some model-> ResNet50  + new full-connected layer (feature vector-> probability that image belongs to one of n classes) (Done-ish)
5. Additional training with the above model minimizing the loss wrt. actual rotated angle
f(I_A)=min[L(A_predict,A)]
6. Save this new model

7. Proof of concept:
Imagenet-> Train on Caltech validation on caltech




Definition of Done: ...

# Imports


In [1]:
# Mount Google drive
from google.colab import drive
import os
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
# Change to current dataset
os.chdir("/content/gdrive/Shared drives/2020_FDLUSA_Earth Science_Knowledge Discovery Framework/Code")

In [3]:
# Imports from Colab 2
import math
import numpy as np
import pickle
import keras
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Import model architecture
from tensorflow.keras.applications import VGG16

Using TensorFlow backend.


In [4]:
# Imports for Colab 6
import cv2 # Read raw image
import glob
from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt
from scipy import ndimage # For rotation task or
import imutils
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

from tensorflow.python.keras.utils import data_utils
from tensorflow.keras.preprocessing.image import Iterator


# Imports for Colorizer
from os import path
from tensorflow.keras import Input
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, UpSampling2D, AveragePooling2D, MaxPooling2D, Reshape, Conv2DTranspose, ZeroPadding2D, Add
from tensorflow.keras.layers import Activation, InputLayer, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray
from skimage.io import imsave
import random
import tensorflow as tf
from skimage.transform import resize
from tensorflow.keras.layers import PReLU



In [5]:
# Check to see if GPU is being used
tensorflow.test.gpu_device_name()

'/device:GPU:0'

# Data Augmentation/Analysis

In [6]:
# Raw dataset/path/model paths
dataset = "UCMerced_LandUse"
dataPath = ("Datasets/"+ dataset+ "/Splits_2")

train_image_globs=glob.glob(dataPath+'/train/**/*.tif')
val_image_globs=glob.glob(dataPath+'/val/**/*.tif')

# Dataloader creation and test

In [7]:
dims=(448,448,3)

train_name='merced_xtrain.npy'
test_name='merced_xtest.npy'

if path.exists(train_name):
  print("Train exists")
  X_train=np.load(train_name)
else:
  X_train=np.empty((0,*dims))

  # for f in train_image_globs:
  #   image=np.expand_dims(img_to_array(load_img(f,target_size=dims))/255.,0)
  #   X_train=np.vstack((X_train,image))

  # with open(train_name, 'wb') as f:
  #   np.save(f,X_train,allow_pickle=True)
  

if path.exists(test_name):
  print("Test exists")
  X_test=np.load(test_name)
else:
  X_test=np.empty((0,*dims))

#   for f in val_image_globs:
#     image=np.expand_dims(img_to_array(load_img(f,target_size=dims))/255.,0)
#     X_test=np.vstack((X_test,image))

#   with open(test_name, 'wb') as f:
#     np.save(f,X_test,allow_pickle=True)

Train exists
Test exists


In [8]:

print(X_train.shape,X_test.shape)

(1680, 256, 256, 3) (420, 256, 256, 3)


In [9]:
train_dataGenerator = ImageDataGenerator(rotation_range=20,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   zoom_range=0.1,
                                   horizontal_flip=True)
valid_dataGenerator = ImageDataGenerator()
batch_size = 16



In [10]:

# def batch_lab(batch_size,channel_pred):
#   for batch in train_dataGenerator.flow(X_train,batch_size=batch_size):
#     lab_batch=rgb2lab(batch)

#     if channel_pred=="0":
#       X_batch=lab_batch[:,:,:,[0]]
#       Y_batch=lab_batch[:,:,:,[1,2]]
#     elif channel_pred=="1_2":
#       X_batch=lab_batch[:,:,:,[1,2]]
#       Y_batch=lab_batch[:,:,:,[0]]
#     yield X_batch,Y_batch

def batch_lab(batch_size,data_generator,data): # Does basically nothing, but just to help with later tasks
  for batch in data_generator.flow(data,batch_size=batch_size):
    # print(batch.shape)
    batch=resize(batch,(batch_size,*dims))
    # print(batch.shape)
    yield batch,batch

In [11]:
def custom_loss(y_true, y_pred):
  y_true_hist=tf.histogram_fixed_width(y_true, value_range=(-1., 1.), nbins=20) 
  y_pred_hist=tf.histogram_fixed_width( y_pred, value_range=(-1., 1.), nbins=20) 
  return K.mean(K.square(y_true_hist-y_pred_hist)) + y_pred*0 

# Model creation

In [12]:
def network(input_shape):

    model = Sequential(name="enc_dec")
    model.add(Input(shape=input_shape))
    model.add(Conv2D(32, (3, 3), padding="same", strides=2))
    model.add(PReLU())
    model.add(Conv2D(64, (3, 3), padding="same"))
    model.add(PReLU())
    model.add(Conv2D(64, (3, 3), padding="same", strides=2))
    model.add(PReLU())
    model.add(Conv2D(128, (3, 3), padding="same"))
    model.add(PReLU())
    model.add(Conv2D(128, (3, 3), padding="same", strides=2))
    model.add(PReLU())
    model.add(Conv2D(256, (3, 3), padding="same"))
    model.add(PReLU())
    model.add(Conv2D(256, (3, 3), padding="same", strides=2))
    model.add(PReLU())
    model.add(Conv2D(15, (3, 3), padding="same"))
    model.add(PReLU())
    model.add(Conv2D(5, (3, 3), padding="same"))
    model.add(PReLU())
    model.add(Conv2DTranspose(128, (3, 3), padding="same"))
    model.add(PReLU())
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2DTranspose(64, (3, 3),padding="same"))
    model.add(PReLU())
    model.add(Conv2DTranspose(64, (3, 3), padding="same"))
    model.add(PReLU())
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2DTranspose(32, (3, 3), padding="same"))
    model.add(PReLU())
    model.add(Conv2D(3, (3, 3), padding="same"))
    model.add(PReLU())
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2DTranspose(3, (3, 3), padding="same"))
    model.add(PReLU())
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2DTranspose(3, (3, 3), activation="tanh", padding="same"))
    return model




# def decoder(input_shape):
#     model = Sequential(name="decoder")
#     model.add(Input(shape=input_shape))
#     model.add(Conv2DTranspose(128, (3, 3), padding="same"))
#     model.add(PReLU())
#     model.add(UpSampling2D((2, 2)))
#     model.add(Conv2DTranspose(64, (3, 3),padding="same"))
#     model.add(PReLU())
#     model.add(Conv2DTranspose(64, (3, 3), padding="same"))
#     model.add(PReLU())
#     model.add(UpSampling2D((2, 2)))
#     model.add(Conv2DTranspose(32, (3, 3), padding="same"))
#     model.add(PReLU())
#     model.add(Conv2D(3, (3, 3), padding="same"))
#     model.add(PReLU())
#     model.add(UpSampling2D((2, 2)))
#     model.add(Conv2DTranspose(3, (3, 3), padding="same"))
#     model.add(PReLU())
#     model.add(UpSampling2D((2, 2)))
#     model.add(Conv2DTranspose(3, (3, 3), activation="tanh", padding="same"))
#     return model


# encoder_model=encoder(dims)
# encoding_depth=15

# print(encoder_model.summary())
# decoder_model=decoder(encoder_model.output_shape[1:])
# print(decoder_model.summary())

complete_model = network(input_shape=dims)

# complete_model.build(input_shape=(None,*dims))
# print(complete_model.summary())



# Model Training

In [13]:
complete_model.compile(optimizer='rmsprop', loss='mse')
complete_model.summary()

Model: "enc_dec"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 32)      896       
_________________________________________________________________
p_re_lu (PReLU)              (None, 224, 224, 32)      1605632   
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      18496     
_________________________________________________________________
p_re_lu_1 (PReLU)            (None, 224, 224, 64)      3211264   
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 64)      36928     
_________________________________________________________________
p_re_lu_2 (PReLU)            (None, 112, 112, 64)      802816    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     7385

In [14]:
!pip install tf-explain
from tf_explain.callbacks.activations_visualization import ActivationsVisualizationCallback
# Define the Activation Visualization callback
output_dir = './visualizations'
callbacks = [
    ActivationsVisualizationCallback(
        validation_data=(X_test, X_test),
        layers_name=['conv2d_8'],
        output_dir=output_dir,
    ),
]

In [15]:
complete_model.fit_generator(batch_lab(batch_size,train_dataGenerator,X_train),
                    epochs=50,
                    steps_per_epoch=len(X_train)/batch_size,
                    validation_data=batch_lab(batch_size,valid_dataGenerator,X_test),
                    validation_steps=len(X_test)/batch_size,
                    callbacks=callbacks)


Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50
105/105 [==============================] - ETA: 0s - loss: 0.0686WARNING:tensorflow:Model was constructed with shape (None, 448, 448, 3) for input Tensor("input_1:0", shape=(None, 448, 448, 3), dtype=float32), but it was called on an input with incompatible shape (None, 256, 256, 3).


ValueError: ignored

In [ ]:
model.save('Models/Colorization_CustomCNN_epoch30_ucmerced')

## Model Testing

In [14]:
model=load_model('Models/Colorization_CustomCNN_epoch30_ucmerced')

In [ ]:
for i in range(10):
  index=np.random.randint(0,len(X_test))

  X_test_im=rgb2gray(np.expand_dims(X_test[index],0))
  X_test_im=np.repeat(np.expand_dims(X_test_im,3),3,axis=3)

  out_image=np.squeeze(model.predict(X_test_im))

  fig=plt.figure()
  plt.subplot(1, 3, 1)
  plt.imshow(X_test[index])
  plt.subplot(1,3,2)
  plt.imshow(np.squeeze(X_test_im))
  plt.subplot(1,3,3)
  plt.imshow(out_image)
  plt.show()

  
  

In [ ]:
X_test_m = rgb2lab(X_test)[:,:,:,0]
X_test_m = X_test_m.reshape(X_test_m.shape+(1,))
Y_test = rgb2lab(X_test)[:,:,:,1:]
Y_test = Y_test / 128
print(model.evaluate(X_test_m, Y_test, batch_size=batch_size))

In [ ]:
rand_index=np.random.randint(0,len(val_image_globs))

sample_image=img_to_array(load_img(val_image_globs[rand_index]))
sample_image=np.expand_dims(sample_image/255.,0)

sample_image_x=rgb2lab(sample_image)[:,:,:,0]
sample_image_x=sample_image_x.reshape(sample_image_x.shape+(1,))

# Test model
output = model.predict(sample_image_x)
output = output * 128

# Output colorizations
for i in range(len(output)):
    cur = np.zeros((256, 256, 3))
    cur[:,:,0] = sample_image_x[i][:,:,0]
    cur[:,:,1:] = output[i]
    imsave("Models/colorization_out/img_"+str(rand_index)+".png", lab2rgb(cur))
    imsave("Models/colorization_out/raw_img_"+str(rand_index)+".png",np.squeeze(sample_image))

